In [17]:
import pandas as pd
import glob

# LME

In [10]:
df = pd.read_csv('../LLLT-mixed_linear_effects.csv')

In [11]:
# drop last 4 columns
df = df.iloc[:,:-4]

# rename for consistency
df = df.rename({'task_difficulty_pre': 'task_difficultyPre', 'task_difficulty_post': 'task_difficultyPost'}, axis=1)

In [13]:
df

,sid,mci_status,tx_allocation,age,sex,education,task_difficultyPre,task_difficultyPost,LowBackCorrectPre,MidBackCorrectPre,HiBackCorrectPre,LowBackRTPre,MidBackRTPre,HiBackRTPre,LowBackCorrectPost,MidBackCorrectPost,HiBackCorrectPost,LowBackRTPost,MidBackRTPost,HiBackRTPost
0,2006,0,0,73.847,1,14.0,1.0,0.0,41.67,38.10,24.36,1398.01,1275.70,1512.73,38.89,36.90,32.05,1341.18,1166.58,1429.63
1,2007,0,1,70.620,0,16.0,0.0,0.0,44.44,46.43,33.33,790.33,806.11,864.13,41.67,47.62,34.62,991.72,993.08,1032.85
2,2008,0,0,72.036,1,22.0,0.0,0.0,16.67,17.86,16.67,874.25,824.02,764.00,38.89,34.52,35.90,1011.14,1193.67,878.32
3,2009,0,1,72.135,1,16.0,0.0,0.0,48.61,29.76,37.18,703.66,808.82,628.61,47.22,35.71,41.03,555.56,533.06,513.77
4,2010,0,1,73.458,1,14.0,1.0,0.0,31.94,35.71,16.67,642.60,1401.92,1595.57,41.67,46.43,34.62,1062.95,1243.14,1196.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,9109,1,1,62.682,1,18.0,1.0,1.0,100.00,51.19,64.10,1096.86,1787.01,1520.47,97.22,45.24,58.97,1080.23,1478.85,1417.34
86,9110,1,1,61.184,1,22.0,1.0,1.0,98.61,100.00,76.92,596.39,747.64,1239.28,100.00,100.00,73.08,625.31,768.98,1399.75
87,9112,1,0,56.987,1,14.0,1.0,1.0,100.00,90.48,78.21,615.32,683.91,759.14,100.00,90.48,88.46,545.81,603.70,735.73
88,9113,1,1,67.365,1,22.0,1.0,1.0,88.89,76.19,46.15,1032.07,1640.89,1697.02,95.83,80.95,69.23,1295.31,1763.37,1856.11


In [16]:
df_long = pd.wide_to_long(df,
                i = ['sid', 'mci_status', 'tx_allocation', 'age', 'sex', 'education'],
                stubnames=[ 'task_difficulty', 'LowBackCorrect', 'MidBackCorrect', 'HiBackCorrect', 'LowBackRT', 'MidBackRT', 'HiBackRT'],
                sep='', suffix='\w+',
                j = 'time' )

task_difficulty  \
sid  mci_status tx_allocation age    sex education time                    
2006 0          0             73.847 1   14.0      Pre               1.0   
                                                   Post              0.0   
2007 0          1             70.620 0   16.0      Pre               0.0   
                                                   Post              0.0   
2008 0          0             72.036 1   22.0      Pre               0.0   
...                                                                  ...   
9112 1          0             56.987 1   14.0      Post              1.0   
9113 1          1             67.365 1   22.0      Pre               1.0   
                                                   Post              1.0   
9116 1          1             66.039 1   NaN       Pre               0.0   
                                                   Post              0.0   

                                                         LowBackCorrect  \
sid  mci_status tx_allocation age    sex education time                   
2006 0          0             73.847 1   14.0      Pre            41.67   
                                                   Post           38.89   
2007 0          1             70.620 0   16.0      Pre            44.44   
                                                   Post           41.67   
2008 0          0             72.036 1   22.0      Pre            16.67   
...                                                                 ...   
9112 1          0             56.987 1   14.0      Post          100.00   
9113 1          1             67.365 1   22.0      Pre            88.89   
                                                   Post           95.83   
9116 1          1             66.039 1   NaN       Pre            34.72   
                                                   Post           43.06   

                                                         MidBackCorrect  \
sid  mci_status tx_allocation age    sex education time                   
2006 0          0             73.847 1   14.0      Pre            38.10   
                                                   Post           36.90   
2007 0          1             70.620 0   16.0      Pre            46.43   
                                                   Post           47.62   
2008 0          0             72.036 1   22.0      Pre            17.86   
...                                                                 ...   
9112 1          0             56.987 1   14.0      Post           90.48   
9113 1          1             67.365 1   22.0      Pre            76.19   
                                                   Post           80.95   
9116 1          1             66.039 1   NaN       Pre            33.33   
                                                   Post           48.81   

                                                         HiBackCorrect  \
sid  mci_status tx_allocation age    sex education time                  
2006 0          0             73.847 1   14.0      Pre           24.36   
                                                   Post          32.05   
2007 0          1             70.620 0   16.0      Pre           33.33   
                                                   Post          34.62   
2008 0          0             72.036 1   22.0      Pre           16.67   
...                                                                ...   
9112 1          0             56.987 1   14.0      Post          88.46   
9113 1          1             67.365 1   22.0      Pre           46.15   
                                                   Post          69.23   
9116 1          1             66.039 1   NaN       Pre           25.64   
                                                   Post          33.33   

                                                         LowBackRT  MidBackRT  \
sid  mci_status tx_allocation age    sex education time                 

In [24]:
# get input ppi data

ppi_list = glob.glob('/scratch/05201/bhickson/preprocdata/ppi/connected_regions_0005/*/thresh_zstat3.nii.gz')


In [29]:
ppi_list[0]

'/scratch/05201/bhickson/preprocdata/ppi/connected_regions_0005/sub-9017_ses-B_run-02.feat/thresh_zstat3.nii.gz'

In [ ]:
for i in ppi_list:
    sub=str(file.split('/')[7].split('_')[0].split('-')[1])
    ses=file.split('/')[7].split('_')[1].split('-')[1]
    run=int(file.split('/')[10].split('_')[3].split('-')[1])